In [1]:
from model import *
from data import *

Using TensorFlow backend.
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/srv/conda/envs/notebook/lib/python3.7/site-packag

## Train your Unet with membrane data
membrane data is in folder membrane/, it is a binary classification task.

The input shape of image and mask are the same :(batch_size,rows,cols,channel = 1)

### Train with data generator

In [2]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(2,'/data/spacenet/bldg/AOI_2_Vegas_Train','PAN-PNG','GT-PNG',data_gen_args,save_to_dir = None)
model = unet()
model_checkpoint = ModelCheckpoint('unet_membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/home/dspuser/unet/model.py:55: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  model = Model(input = inputs, output = conv10)



Epoch 1/5
Found 3851 images belonging to 1 classes.
Found 3851 images belonging to 1 classes.
2000/2000 [==============================] - 162s 81ms/step - loss: 0.3317 - accuracy: 0.8707

Epoch 00001: loss improved from inf to 0.33174, saving model to unet_membrane.hdf5
Epoch 2/5
2000/2000 [==============================] - 145s 73ms/step - loss: 0.1790 - accuracy: 0.9294

Epoch 00002: loss improved from 0.33174 to 0.17899, saving model to unet_membrane.hdf5
Epoch 3/5
2000/2000 [==============================] - 146s 73ms/step - loss: 0.1553 - accuracy: 0.9393

Epoch 00003: loss improved from 0.17899 to 0.15532, saving model to unet_membrane.hdf5
Epoch 4/5
2000/2000 [==============================] - 147s 73ms/step - loss: 0.1444 - accuracy: 0.9440

Epoch 00004: loss improved from 0.15532 to 0.14443, saving model to unet_membrane.hdf5
Epoch 5/5
2000/2000 [==============================] - 146s 73ms/step - loss: 0.1321 - accuracy: 0.9481

Epoch 00005: loss improved from 0.14443 to 0.1

### Train with npy file

In [3]:
#imgs_train,imgs_mask_train = geneTrainNpy("data/membrane/train/aug/","data/membrane/train/aug/")
#model.fit(imgs_train, imgs_mask_train, batch_size=2, nb_epoch=10, verbose=1,validation_split=0.2, shuffle=True, callbacks=[model_checkpoint])

### test your model and save predicted results

In [11]:
from PIL import Image
import numpy as np
from skimage import transform

In [12]:
#testGene = testGenerator("/data/spacenet/bldg/AOI_2_Vegas_Test_public/PAN-PNG")
model = unet()
model.load_weights("unet_membrane.hdf5")
np_image = Image.open('/data/spacenet/bldg/AOI_2_Vegas_Test_public/PAN-PNG/PAN_AOI_2_Vegas_img1005.png')
np_image = np.array(np_image).astype('float32')/255
np_image = transform.resize(np_image, (256, 256, 1))
np_image = np.expand_dims(np_image, axis=0)


#model.predict("/data/spacenet/bldg/AOI_2_Vegas_Test_public/PAN-PNG/PAN_AOI_2_Vegas_img1005.png")
#results = model.predict_generator(testGene,30,verbose=1)
#saveResult("data/membrane/test2",results)

In [15]:
im = model.predict(np_image)

In [16]:
import matplotlib.pyplot as plt
plt.imshow(im)

TypeError: Invalid shape (1, 256, 256, 1) for image data